# Urban Centers at Flood Risk Analysis

This notebook guides you through:

1. Selecting an Area of Interest (AOI) interactively on a map.
2. Fetching demographic and flood risk data.
3. Identifying urban centers within the AOI that are at the highest risk due to flooding, using GHS settlement population metrics.
4. Visualizing and ranking these areas.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/worldbank/DECAT_Space2Stats/blob/main/docs/user-docs/space2stats_api_demo_urban_flood_risk.ipynb)

In [ ]:
# !pip install geopandas ipyleaflet

In [1]:
import json
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from ipyleaflet import Map, DrawControl
from space2stats_client import AOISelector, Space2StatsClient

In [40]:
widget, aoi = AOISelector(center=(27.0, 29.7), zoom=6)
display(widget)

In [41]:
aoi

                                            geometry        name
0  POLYGON ((29.42139 24.54313, 29.42139 30.29322...  User AOI 1

In [42]:
client=Space2StatsClient()
fields = [
    "sum_pop_2020", "sum_pop_f_2020", "sum_pop_m_2020", "pop_flood", "pop_flood_pct",
    "ghs_11_pop", "ghs_12_pop", "ghs_13_pop", "ghs_21_pop", "ghs_22_pop", "ghs_23_pop", "ghs_30_pop"
]

In [54]:
# Get available topics/datasets
df = client.get_summary(
    gdf=aoi.gdf,
    spatial_join_method="centroid",
    fields=fields,
    geometry="polygon"
)

Fetching data for boundary 1 of 1...


In [44]:
# Convert df to gdf
if isinstance(df.geometry.iloc[0], str):
    df["geometry"] = df.geometry.apply(json.loads)
df["geometry"] = df.geometry.apply(shape)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

In [46]:
# Define urban_pop to only include semi-dense urban clusters (22_POP), dense urban clusters (23_POP) and urban centres (30_POP)
gdf["urban_pop"] = gdf["ghs_22_pop"] + gdf["ghs_23_pop"] + gdf["ghs_30_pop"]

# Calculate risk score using only the updated urban_pop
gdf["risk_score"] = gdf["pop_flood_pct"] * gdf["urban_pop"]

# Filter to areas where urban_pop is significant (e.g., > 100 people)
urban_gdf = gdf[gdf["urban_pop"] > 100].copy()

max_score = urban_gdf["risk_score"].max()
urban_gdf["risk_score_norm"] = (urban_gdf["risk_score"] / max_score) * 100

# Now sorting and other operations will not raise SettingWithCopyWarning
urban_gdf = urban_gdf.sort_values("risk_score_norm", ascending=False)
urban_gdf.head()


,name,index_gdf,index_h3,hex_id,geometry,sum_pop_2020,sum_pop_f_2020,sum_pop_m_2020,pop_flood,pop_flood_pct,ghs_11_pop,ghs_12_pop,ghs_13_pop,ghs_21_pop,ghs_22_pop,ghs_23_pop,ghs_30_pop,urban_pop,risk_score,risk_score_norm
3895,User AOI 1,0,3895,863e5ca5fffffff,"POLYGON ((32.61273 25.74394, 32.59535 25.71109...",1.593942e+10,7.927194e+09,8.012227e+09,859140.449547,0.801047,0.000000,154.433368,0.0,0.000000,0.0,0.000000,1.338677e+06,1.338677e+06,1.072343e+06,100.000000
2076,User AOI 1,0,2076,863e46657ffffff,"POLYGON ((31.14573 27.21547, 31.12845 27.18234...",1.593942e+10,7.927194e+09,8.012227e+09,368488.695989,0.650426,0.286214,43.898725,0.0,915.403993,0.0,20.302995,6.416322e+05,6.416525e+05,4.173473e+05,38.919187
4270,User AOI 1,0,4270,863e5ddb7ffffff,"POLYGON ((32.61781 25.68118, 32.60043 25.64833...",1.593942e+10,7.927194e+09,8.012227e+09,326864.062943,0.937185,14.670358,897.452911,0.0,2812.929213,0.0,0.000000,4.436752e+05,4.436752e+05,4.158056e+05,38.775416
4798,User AOI 1,0,4798,863e6280fffffff,"POLYGON ((31.2357 30.14413, 31.21791 30.11113,...",1.593942e+10,7.927194e+09,8.012227e+09,341985.580521,0.149515,0.000000,0.000000,0.0,0.000000,0.0,0.000000,2.668585e+06,2.668585e+06,3.989945e+05,37.207717
3893,User AOI 1,0,3893,863e5ca4fffffff,"POLYGON ((32.67 25.77969, 32.65259 25.74685, 3...",1.593942e+10,7.927194e+09,8.012227e+09,392636.870454,0.957646,0.000000,118.216947,0.0,3588.559773,0.0,0.000000,3.132943e+05,3.132943e+05,3.000252e+05,27.978462


In [52]:
# Now create your risk map
m_risk = urban_gdf.explore(
    column="risk_score_norm",
    tooltip=["sum_pop_2020", "pop_flood", "pop_flood_pct", "urban_pop", "risk_score"],
    cmap="OrRd",
    legend=True,
    scheme="quantiles",
    legend_kwds=dict(colorbar=True, caption="Urban Flood Score", interval=False),
    style_kwds=dict(weight=0.5, fillOpacity=0.8),
    name="Urban Flood Risk",
)

aoi.gdf.explore(
    m=m_risk,  # Add to the existing map
    color='red',
    weight=3,
    fill=False,
    name="AOI Boundary"
)

m_risk